In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#format for floats
pd.options.display.float_format = '{:,.2f}'.format

In [16]:
df0 = pd.read_csv('../Neueda2023/Datasets/2010.csv')
df1 = pd.read_csv('../Neueda2023/Datasets/2011.csv')
df2 = pd.read_csv('../Neueda2023/Datasets/2012.csv')
df3 = pd.read_csv('../Neueda2023/Datasets/2013.csv')
df4 = pd.read_csv('../Neueda2023/Datasets/2014.csv')
df5 = pd.read_csv('../Neueda2023/Datasets/2015.csv')
df6 = pd.read_csv('../Neueda2023/Datasets/2016.csv')

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2010,274,188,80,6,0,0,0,150,71,43,0,0,192,82,0
1,Alabama,Clay,2010,113,79,34,0,0,0,0,86,60,42,0,0,0,113,0
2,Alabama,Colbert,2010,355,208,146,1,0,0,0,105,67,47,0,0,114,241,0
3,Alabama,DeKalb,2010,358,248,106,4,0,0,0,101,74,43,0,0,303,55,0
4,Alabama,Elmore,2010,241,181,55,5,0,0,0,112,71,43,0,0,241,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,Wyoming,Sublette,2016,366,297,68,1,0,0,0,102,61,44,0,0,351,15,0
1026,Wyoming,Sweetwater,2016,366,304,62,0,0,0,0,100,58,44,0,0,327,4,35
1027,Wyoming,Teton,2016,366,331,35,0,0,0,0,99,50,41,0,0,351,15,0
1028,Wyoming,Uinta,2016,366,341,25,0,0,0,0,87,49,42,0,0,359,0,7


In [24]:
df0['Average Max AQI'] = df0.groupby(by = 'State')['Max AQI'].mean()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,Average Max AQI
0,Alabama,Baldwin,2010,274,188,80,6,0,0,0,150,71,43,0,0,192,82,0,112.26
1,Alabama,Clay,2010,113,79,34,0,0,0,0,86,60,42,0,0,0,113,0,112.26
2,Alabama,Colbert,2010,355,208,146,1,0,0,0,105,67,47,0,0,114,241,0,112.26
3,Alabama,DeKalb,2010,358,248,106,4,0,0,0,101,74,43,0,0,303,55,0,112.26
4,Alabama,Elmore,2010,241,181,55,5,0,0,0,112,71,43,0,0,241,0,0,112.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,Wyoming,Sheridan,2010,365,333,32,0,0,0,0,95,46,21,0,0,0,97,268,112.26
1072,Wyoming,Sublette,2010,365,250,113,2,0,0,0,112,67,47,0,2,347,16,0,112.26
1073,Wyoming,Sweetwater,2010,365,255,107,3,0,0,0,112,66,45,0,6,314,7,38,112.26
1074,Wyoming,Teton,2010,365,281,84,0,0,0,0,90,61,44,4,0,355,4,2,112.26


In [17]:
df_aqi.groupby(by = 'State', 'Year')['Max AQI'].mean()

SyntaxError: positional argument follows keyword argument (<ipython-input-17-b723bc6f3755>, line 1)